In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader

# === Step 1: Load and preprocess dataset ===
#df = pd.read_csv("cleaned_weather_dataset.csv")
#df = pd.read_csv(r"C:\Users\DELL\main\cleaned_weather_dataset.csv")
df = pd.read_csv(r"C:\Users\DELL\main\cleaned_weather_dataset.csv")

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

features = ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT08']
data = df[features].values

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

INPUT_DAYS = 14
PREDICT_DAYS = 7

# === Step 2: Create Dataset ===
class WeatherDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data) - INPUT_DAYS - PREDICT_DAYS
    def __getitem__(self, idx):
        x = self.data[idx:idx+INPUT_DAYS]
        y = self.data[idx+INPUT_DAYS:idx+INPUT_DAYS+PREDICT_DAYS]
        return torch.FloatTensor(x), torch.FloatTensor(y)

dataset = WeatherDataset(scaled_data)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# === Step 3: Define LSTM model ===
class LSTMForecast(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.encoder = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h, c) = self.encoder(x)
        decoder_input = x[:, -1:, :]
        outputs = []
        for _ in range(PREDICT_DAYS):
            out, (h, c) = self.decoder(decoder_input, (h, c))
            step_output = self.output_layer(out[:, -1, :])
            outputs.append(step_output.unsqueeze(1))
            decoder_input = step_output.unsqueeze(1)
        return torch.cat(outputs, dim=1)

# === Step 4: Train the model ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMForecast(input_size=11, hidden_size=64, output_size=11).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

print("🧠 Starting training...")
for epoch in range(20):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = criterion(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} | Loss: {total_loss / len(loader):.5f}")

# === Step 5: Save model weights ===
torch.save(model.state_dict(), "lstm_multivariate_7day.pth")
print("✅ Model trained and saved as lstm_multivariate_7day.pth")


In [ ]:
import os
print(os.getcwd())


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

# === Step 1: Load dataset ===
df = pd.read_csv("cleaned_weather_dataset.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')
df = df.fillna(df.mean(numeric_only=True))

features = ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT08']
data = df[features].values
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)

INPUT_DAYS = 14
PREDICT_DAYS = 7

# === Step 2: Dataset class ===
class WeatherDataset(Dataset):
    def __init__(self, sequence):
        self.sequence = sequence
    def __len__(self):
        return len(self.sequence) - INPUT_DAYS - PREDICT_DAYS
    def __getitem__(self, idx):
        x = self.sequence[idx:idx+INPUT_DAYS]
        y = self.sequence[idx+INPUT_DAYS:idx+INPUT_DAYS+PREDICT_DAYS]
        return torch.FloatTensor(x), torch.FloatTensor(y)

dataset = WeatherDataset(scaled)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# === Step 3: Define LSTM model ===
class LSTMForecast(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.encoder = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h, c) = self.encoder(x)
        decoder_input = x[:, -1:, :]
        outputs = []
        for _ in range(PREDICT_DAYS):
            out, (h, c) = self.decoder(decoder_input, (h, c))
            step = self.output_layer(out[:, -1, :])
            outputs.append(step.unsqueeze(1))
            decoder_input = step.unsqueeze(1)
        return torch.cat(outputs, dim=1)

# === Step 4: Train ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMForecast(input_size=11, hidden_size=64, output_size=11).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

print("🔄 Starting training...")
for epoch in range(20):
    total_loss = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} | Loss: {total_loss / len(loader):.5f}")

# === Step 5: Save weights ===
torch.save(model.state_dict(), "lstm_multivariate_7day.pth")
print("✅ Model trained and saved as lstm_multivariate_7day.pth")


In [2]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

# Load cleaned forecast data
#df = pd.read_csv("forecast_7day.csv")  # replace with your actual filename
df = pd.read_csv(r"C:\Users\DELL\main\cleaned_weather_dataset.csv")

st.title("🌦️ 7-Day Weather Forecast Dashboard")

st.dataframe(df.style.format(precision=2))

# Plot each feature
features = ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT08']
selected = st.multiselect("Select features to plot:", features, default=features[:3])

for feature in selected:
    st.subheader(f"{feature} - 7-Day Forecast")
    fig, ax = plt.subplots()
    ax.plot(df['Day'], df[feature], marker='o')
    ax.set_title(f"{feature} over the Next 7 Days")
    ax.set_xlabel("Day")
    ax.set_ylabel(feature)
    ax.grid(True)
    st.pyplot(fig)


2025-04-29 19:59:22.921 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 19:59:22.963 
  command:

    streamlit run C:\Users\DELL\anaconda3\envs\ecc-env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-29 19:59:22.963 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 19:59:23.045 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 19:59:23.112 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 19:59:23.112 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 19:59:23.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 19:59:23.115 Thread 'Mai

KeyError: 'Day'

In [3]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

# === App Title ===
st.title("📊 7-Day Weather Forecast Dashboard")
st.markdown("This dashboard visualizes forecasted weather data from your cleaned dataset.")

# === Load Data ===
uploaded_file = st.file_uploader("📁 Upload the forecast CSV file", type=["csv"])
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)

    st.success("✅ File loaded successfully!")

    # === Show Columns ===
    st.subheader("📋 Columns in the dataset")
    st.write(df.columns.tolist())

    # === Check for 'Day' or 'Date' column ===
    if 'Day' in df.columns:
        x_axis = df['Day']
        x_label = "Day"
    elif 'Date' in df.columns:
        x_axis = df['Date']
        x_label = "Date"
    else:
        st.error("❌ Neither 'Day' nor 'Date' column is present in the dataset.")
        st.stop()

    # === Feature Columns ===
    excluded_columns = ['Day', 'Date']
    features = [col for col in df.columns if col not in excluded_columns]

    # === Plot each feature ===
    for feature in features:
        st.subheader(f"📈 {feature} — 7-Day Forecast")
        fig, ax = plt.subplots()
        ax.plot(x_axis, df[feature], marker='o', linestyle='-')
        ax.set_xlabel(x_label)
        ax.set_ylabel(feature)
        ax.set_title(f"{feature} over the Next 7 Days")
        ax.grid(True)
        plt.xticks(rotation=45)
        st.pyplot(fig)

else:
    st.warning("📂 Please upload a forecasted CSV file to begin.")


2025-04-29 20:01:07.786 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 20:01:07.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 20:01:07.790 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 20:01:07.790 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 20:01:07.791 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 20:01:07.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 20:01:07.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-29 20:01:07.795 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar